In [1]:

import torch
import os
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.models as models
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader
import random

# Set the seed for reproducibility
SEED = 42
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)  # Apply the seed

In [ ]:
# Define the categories and the paths to datasets
checkpoint_path = r'C:\Users\Josh\Desktop\CUDA\52.41.pth'

BATCH_SIZE = 32
INPUT_SIZE = (256, 256)

def count_files_in_directory(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files

main_data_dir = r"C:\Users\Josh\Desktop\CUDA\splitdata"
train_dir = os.path.join(main_data_dir, "train")
val_dir = os.path.join(main_data_dir, "val")
test_dir = os.path.join(main_data_dir, "test")

train_files = count_files_in_directory(train_dir)
val_files = count_files_in_directory(val_dir)
test_files = count_files_in_directory(test_dir)

print(f"Training Dataset: {train_files}")
print(f"Validation Dataset: {val_files}")
print(f"Test Dataset: {test_files}")

In [ ]:

# Load the training dataset to calculate mean and std, and get class labels
train_dataset = datasets.ImageFolder(root=train_dir)
class_n = list(train_dataset.class_to_idx.keys())  # Automatically retrieves class names from folders
print("Class to label mapping:", train_dataset.class_to_idx)

In [4]:
import cv2
import numpy as np
from torchvision import transforms
from PIL import Image

MEAN = (0.6181, 0.4643, 0.4194)
STD = (0.1927, 0.1677, 0.1617)


class CLAHETransform:
    def __init__(self, clip_limit=2.0, tile_grid_size=(8, 8)):
        self.clip_limit = clip_limit
        self.tile_grid_size = tile_grid_size

    def __call__(self, img):
        # Convert PIL Image to NumPy array
        img_np = np.array(img)
        
        # Apply CLAHE on each channel independently if it's a color image
        if len(img_np.shape) == 3:
            channels = cv2.split(img_np)
            clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
            channels = [clahe.apply(channel) for channel in channels]
            img_np = cv2.merge(channels)
        else:
            # Apply CLAHE on grayscale images
            clahe = cv2.createCLAHE(clipLimit=self.clip_limit, tileGridSize=self.tile_grid_size)
            img_np = clahe.apply(img_np)

        # Convert back to PIL Image
        img_clahe = Image.fromarray(img_np)
        return img_clahe

# Define transformations
transform_train = transforms.Compose([
    # Resize to standardize input dimensions
    transforms.Resize(INPUT_SIZE),
    transforms.CenterCrop((227, 227)), 

    # Apply CLAHE
    transforms.RandomApply([CLAHETransform(clip_limit=2.0, tile_grid_size=(8, 8))], p=0.5),

    # Segmentation and rotations as defined previously
    # KMeansSegmentation(n_clusters=10, p=0.3),
    transforms.RandomApply([transforms.RandomRotation(degrees=(0, 20))], p=0.25),
    transforms.RandomApply([transforms.RandomRotation(degrees=(0, 10))], p=0.25),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-20, 0))], p=0.25),
    transforms.RandomApply([transforms.RandomRotation(degrees=(-10, 0))], p=0.25),

    # Subtle affine transformations and other adjustments
    transforms.RandomApply([transforms.RandomAffine(degrees=5, translate=(0.02, 0.02), shear=2)], p=0.3),
    transforms.RandomApply([transforms.RandomPerspective(distortion_scale=0.05, p=0.2)], p=0.2),
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))], p=0.4),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2)], p=0.7),
    transforms.RandomApply([transforms.RandomGrayscale(p=1.0)], p=0.2),
    transforms.RandomApply([transforms.RandomHorizontalFlip()], p=0.5),
    transforms.RandomApply([transforms.RandomAdjustSharpness(sharpness_factor=1.5)], p=0.3),

    # Convert to tensor and normalize
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

# Validation and test transformations
transform_val_test = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    transforms.CenterCrop((227, 227)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

# Load the datasets with the new transforms
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform_train)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform_val_test)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform_val_test)

# Create DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
# Get a batch of images and labels from the DataLoader
images, labels = next(iter(train_loader))
print(images.shape)  


In [ ]:

# Function to unnormalize the image for visualization
def unnormalize(image, mean, std):
    # Convert the tensor to a NumPy array and transpose dimensions to (H, W, C)
    image = image.numpy().transpose((1, 2, 0))  
    
    # Unnormalize by reversing the mean and std normalization
    image = (image * std) + mean  
    
    # Clip values to be between 0 and 1 for valid image display
    image = np.clip(image, 0, 1)  
    return image


# Visualize a batch of images from the train_loader
def visualize_loader(loader, mean, std, class_names, num_images=6):
    # Get a batch of images
    data_iter = iter(loader)
    images, labels = next(data_iter)  

    # Plot the images
    plt.figure(figsize=(12, 8))
    for i in range(num_images):
        plt.subplot(2, 3, i+1)
        image = unnormalize(images[i], mean, std)  
        plt.imshow(image)
        plt.title(f"Class: {class_names[labels[i]]}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Use this function to visualize a batch of images
visualize_loader(train_loader, mean=MEAN, std=STD, class_names=class_n)

In [ ]:


class AlexNetFC6(nn.Module):
    def __init__(self):
        super(AlexNetFC6, self).__init__()
        alexnet = models.alexnet(pretrained=False)
        self.features = alexnet.features
        self.pooling = nn.AdaptiveAvgPool2d((6, 6))
        self.fc6 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pooling(x) 
        x = torch.flatten(x, 1)
        x = self.fc6(x)
        return x



alexnet_fc6 = AlexNetFC6()
alexnet_fc6.load_state_dict(torch.load(checkpoint_path), strict=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet_fc6.to(device)
alexnet_fc6.eval()  




In [8]:
def extract_features(data_loader, model):
    features = []
    labels = []
    
    with torch.no_grad():  # Disable gradient computation
        for images, target_labels in data_loader:
            images = images.to(device)
            
            # Extract features using the custom AlexNet model (up to fc6)
            output_features = model(images)
            
            # Append extracted features and labels to the list
            features.append(output_features.cpu().numpy())  # Keeping this as CPU NumPy
            labels.append(target_labels.cpu().numpy())
    
    # Concatenate features and labels from all batches
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels


In [9]:
# Extract features from the datasets
train_features, y_train = extract_features(train_loader, alexnet_fc6)
val_features, y_val = extract_features(val_loader, alexnet_fc6)
test_features, y_test = extract_features(test_loader, alexnet_fc6)


In [ ]:
from skopt import BayesSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Define the parameter search space with adjusted learning rate
param_space = {
    'learning_rate': (1e-2, 0.5, 'log-uniform'),  # Adjusted minimum learning rate
    'max_depth': (2, 15),
    'colsample_bytree': (0.3, 1.0, 'uniform'),
    'colsample_bylevel': (0.3, 1.0, 'uniform'),
    'subsample': (0.3, 1.0, 'uniform'),
    'min_child_weight': (1, 50),
    'gamma': (0, 10),
    'reg_alpha': (1e-3, 200, 'log-uniform'),
    'reg_lambda': (1e-3, 200, 'log-uniform'),
    'scale_pos_weight': (0.1, 20.0, 'log-uniform'),
    'max_delta_step': (0, 20),
}

# Initialize XGBClassifier with fixed parameters
model = XGBClassifier(
    n_estimators=1000,
    objective='multi:softmax',
    num_class=len(class_n),
    random_state=SEED,
    early_stopping_rounds=20,
    verbosity=1  # Replace verbose with verbosity to avoid warnings
)

# Bayesian optimization with BayesSearchCV
bayes_cv = BayesSearchCV(
    estimator=model,
    search_spaces=param_space,
    n_iter=20,  # Number of search iterations
    scoring='accuracy',
    cv=5,  # 5-fold cross-validation
    n_jobs=-1,
    random_state=SEED,
    verbose=1
)

# Fit the model using the validation set for early stopping
bayes_cv.fit(
    train_features, y_train,
    eval_set=[(val_features, y_val)],
    verbose=1
)

# Save the best model as JSON
best_model = bayes_cv.best_estimator_
best_model.save_model('best_xgboost_model.json')  # Saves model in JSON format

# Retrieve the best parameters and initialize final model with them for full training
best_params = bayes_cv.best_params_
final_model = XGBClassifier(
    **best_params,
    n_estimators=1000,
    objective='multi:softmax',
    num_class=len(class_n),
    random_state=SEED,
    early_stopping_rounds=20,
    verbosity=1
)

# Fit the final model on training data with validation monitoring
final_model.fit(
    train_features, y_train,
    eval_set=[(train_features, y_train), (val_features, y_val)],
    verbose=1
)

# Save the final model after re-training if desired
final_model.save_model('final_xgboost_model.json')  # Saves model as JSON


In [ ]:
# Evaluate the model on training, validation, and test sets
train_predictions = final_model.predict(train_features)
train_accuracy = accuracy_score(y_train, train_predictions)
print(f'Training Accuracy: {train_accuracy * 100:.2f}%')

val_predictions = final_model.predict(val_features)
val_accuracy = accuracy_score(y_val, val_predictions)
print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

test_predictions = final_model.predict(test_features)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Generate and print classification reports for validation and test sets
val_classification_report = classification_report(y_val, val_predictions, target_names=class_n)
print("Validation Classification Report:\n", val_classification_report)

cm = confusion_matrix(y_val, val_predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_n, yticklabels=class_n)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

test_classification_report = classification_report(y_test, test_predictions, target_names=class_n)
print("Test Classification Report:\n", test_classification_report)

cm = confusion_matrix(y_test, test_predictions)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_n, yticklabels=class_n)

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()
